In [2]:
import numpy as np
import tensorflow as tf
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from transformers import TFViTModel

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Subtract, Input, Flatten, Activation, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

In [8]:
def create_ranking_network(img_size):
    """
    Create ranking network which give a score to an image.

    :param img_size: size of input images during training
    :type img_size: tuple(int)
    :return: ranking network model
    :rtype: keras.Model
    """
    # Create feature extractor from VGG19
    feature_extractor = VGG19(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))
    for layer in feature_extractor.layers[:-4]:
        layer.trainable = False

    # Add dense layers on top of the feature extractor
    inp = Input(shape=(img_size, img_size, 3), name='input_image')
    base = feature_extractor(inp)
    base = Flatten(name='Flatten')(base)

    # Block 1
    base = Dense(32, activation='sigmoid', name='Dense_1')(base)
    base = BatchNormalization(name='BN1')(base)
    base = Dropout(0.2, name='Drop_1')(base)

    # Block 2
    base = Dense(32, activation='sigmoid', name='Dense_2')(base)
    base = BatchNormalization(name='BN2')(base)
    base = Dropout(0.2, name='Drop_2')(base)

    # Final dense
    base = Dense(1, name="Dense_Output")(base)
    base_network = Model(inp, base, name='Scoring_model')
    return base_network

ranking_streetview = create_ranking_network(224)
ranking_streetview.load_weights("../Result/Mapillary_Results/Best_RankingModel_From_Streetview_Trained_On_Mapillary_DataAug_Contrast/ranking_model_weights.h5")


In [6]:
resize_rescale_hf = tf.keras.Sequential([
    tf.keras.layers.Permute((3, 1, 2))
])
def create_ranking_network(img_size):
    """
    Create ranking network which give a score to an image.

    :param img_size: size of input images during training
    :type img_size: tuple(int)
    :return: ranking network model
    :rtype: keras.Model
    """
    # Create feature extractor from VGG19
    feature_extractor = TFViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

    for layer in feature_extractor.layers:
        layer.trainable=True

    # Add dense layers on top of the feature extractor
    inp = Input(shape=(img_size, img_size, 3), name='input_image')
    resized_inp = resize_rescale_hf(inp) 
    base = feature_extractor.vit(resized_inp)[0][:, 0, :]
    base = Flatten(name='Flatten')(base)

    # Block 1
    base = Dense(32, activation='sigmoid', name='Dense_1')(base)
    base = BatchNormalization(name='BN1')(base)
    base = Dropout(0.2, name='Drop_1')(base)

    # Block 2
    base = Dense(32, activation='sigmoid', name='Dense_2')(base)
    base = BatchNormalization(name='BN2')(base)
    base = Dropout(0.2, name='Drop_2')(base)

    # Final dense
    base = Dense(1, name="Dense_Output")(base)
    base_network = Model(inp, base, name='Scoring_model')
    return base_network

ranking_model_transformer =  create_ranking_network(224)
ranking_model_transformer.load_weights("../Result/Transformer_Results/Streetview/Ranking_10E_dataaug_contrast/ranking_model_weights.h5")

All model checkpoint layers were used when initializing TFViTModel.

All the layers of TFViTModel were initialized from the model checkpoint at google/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [8]:
# Chemin vers le dossier contenant les images
image_folder = "../data/question_1/ForPrediction/"

# Initialiser des listes vides pour stocker les données
longitude_list = []
latitude_list = []
score_list = []

# Parcourir les sous-dossiers
for root, dirs, files in os.walk(image_folder):
    for directory in dirs:
        # Extraire la longitude et la latitude du nom du sous-dossier
        longitude, latitude = directory.split(",")
        
        # Chemin vers le sous-dossier
        sub_folder = os.path.join(root, directory)
        
        # Parcourir les fichiers d'images dans le sous-dossier
        for file in os.listdir(sub_folder):
            # Chemin vers l'image
            image_path = os.path.join(sub_folder, file)
            
            # Charger et traiter l'image
            image = cv2.imread(image_path)
            image = cv2.resize(image, (224, 224))
            image = image.astype(np.float32) / 255.0

            image = tf.convert_to_tensor(image)
            
            # Faire la prédiction avec votre modèle et obtenir le score
            score = ranking_model_transformer.predict(np.expand_dims(image, axis=0))
            
            # Ajouter les données aux listes
            longitude_list.append(longitude)
            latitude_list.append(latitude)
            score_list.append(score[0][0])  # Assuming the score is a single value
            
# Créer un DataFrame pandas avec les données
data = {
    "Longitude": longitude_list,
    "Latitude": latitude_list,
    "Score": score_list
}
df = pd.DataFrame(data)

# Enregistrer le DataFrame dans un fichier Excel
output_file = "Transformer_Streetview_Ranking_Predict_On_ForPrediction_Dataset.csv"
df.to_csv(output_file, index=False)
